## Introduction

For this session, we will work on this dataset: [DNA Sequence dataset](https://www.kaggle.com/datasets/nageshsingh/dna-sequence-dataset). 
A more in-depth guide can be found [here](./resources/dna-sequencing-with-machine-learning.ipynb).

Example of fasta file handling using biopython library:

In [7]:
%matplotlib inline
DATASET_PATH = 'resources/dna_dataset/'

In [8]:
from Bio import SeqIO
for sequence in SeqIO.parse(DATASET_PATH + 'example_dna.fa', "fasta"):
    print('Sequence ID:', sequence.id)
    print('Sequence:', sequence.seq)
    print('Sequence length:', len(sequence))
    print('-'*20)
    break

Sequence ID: ENST00000435737.5
Sequence: ATGTTTCGCATCACCAACATTGAGTTTCTTCCCGAATACCGACAAAAGGAGTCCAGGGAATTTCTTTCAGTGTCACGGACTGTGCAGCAAGTGATAAACCTGGTTTATACAACATCTGCCTTCTCCAAATTTTATGAGCAGTCTGTTGTTGCAGATGTCAGCAACAACAAAGGCGGCCTCCTTGTCCACTTTTGGATTGTTTTTGTCATGCCACGTGCCAAAGGCCACATCTTCTGTGAAGACTGTGTTGCCGCCATCTTGAAGGACTCCATCCAGACAAGCATCATAAACCGGACCTCTGTGGGGAGCTTGCAGGGACTGGCTGTGGACATGGACTCTGTGGTACTAAATGAAGTCCTGGGGCTGACTCTCATTGTCTGGATTGACTGA
Sequence length: 390
--------------------


## The dataset

**Objective**: Build a classification model that is trained on the human DNA sequence and can predict a gene family based on the DNA sequence of the coding sequence. To test the model, we will use the DNA sequence of humans, dogs, and chimpanzees and compare the accuracies.

Gene families are groups of related genes that share a common ancestor. Members of gene families may be paralogs or orthologs. Gene paralogs are genes with similar sequences from within the same species while gene orthologs are genes with similar sequences in different species.

The dataset contains human DNA sequence, Dog DNA sequence, and Chimpanzee DNA sequence.

## Analyze the dataset

First, we should understand our dataset and objective, before doing any kind of deep dive.
